In [1]:
import pandas as pd

df = pd.read_json("../../datas/zap/detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 60626}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:41:17.463548,NaN
1,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50540}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.438251,NaN
2,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50540}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.499190,NaN
3,GET,/robots.txt,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50542}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.496824,NaN
4,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50546}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.501211,NaN


In [2]:
methods_matrix = []
tmp = []
sess_uuids_methods = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  method = row["method"]

  if sess_uuid in sess_uuids:
    sess_uuids_methods[sess_uuid].append(method)
  else:
    sess_uuids_methods[sess_uuid] = [method]
    sess_uuids.append(sess_uuid)

methods_matrix = list(sess_uuids_methods.values())
# for paths in sess_uuids_methods.values():
#   methods_matrix.append(paths)

sess_uuids_methods
# methods_matrix
# print(len(methods_matrix), len(sess_uuids_methods.keys()))

{'c58bf9d6-7a8a-4b9c-8dcd-81406de2bbd6': ['GET'],
 'cf902971-14ec-45f8-9839-6fb5e589eb71': ['GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'POST',
  'GET',
  'POST',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'POST',
  'GET',
  'GET',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'GET',
  'POST',
  'POST',
  'POST',
  'POST',
  'POST',
  'POST',
  'POST',
  'GET',
  'GET',
  'GET',
  'GET',

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(methods_matrix).transform(methods_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,GET,POST
0,True,False
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,False


In [ ]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.0000001, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets.head(20)

,support,itemsets
0,1.000000,(GET)
1,0.384615,(POST)
2,0.384615,"(POST, GET)"


In [12]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_zap", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [13]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_methods (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            METHOD            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [14]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_methods (ID, SUPPORT, METHOD)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
